# CNN

## Imports

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision import transforms
from pytorch_lightning import LightningModule, Trainer
from torchvision.datasets import CIFAR10
from pytorch_lightning.callbacks.progress import TQDMProgressBar
import torchmetrics
from pytorch_lightning.loggers import TensorBoardLogger
from IPython.display import display
from pytorch_lightning.callbacks import ModelCheckpoint
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import os
from torch.utils.data import Dataset
import cv2
import os
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
os.environ['TORCH_HOME'] = '.'

In [2]:
#util
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [3]:
def cdf(root_path, folder, label, on_ram):
    folder_files = Path (os.path.join(root_path,folder))
    file_list = []
    file_label = []
    for file_path in sorted(folder_files.glob('*.jpg')):
        
        if on_ram:
            file_list.append(cv2.imread(os.fspath(file_path)))
        else:
            file_list.append(os.fspath(file_path))
        file_label.append(label)
    
    data = {'image':file_list, 'label':file_label}
    
    return pd.DataFrame(data)


class Dataset_Flores(Dataset):
    def __init__(self, root_path = '',transform = None,test=False, on_ram = True ,shuffle = True ):
        self.on_ram = on_ram
        self.transform = transform
        
        
        if test:
            root_path = os.path.join(root_path,'test')
        else:
            root_path = os.path.join(root_path,'train')
        
        
        
        df_Rosas = cdf(root_path, 'Rosas',1,on_ram)
        df_Calas_rosa = cdf(root_path, 'Calas_rosa',2,on_ram)
        df_Cardenales_rojas = cdf(root_path, 'Cardenales_rojas',3,on_ram)
        df_Orejas_de_oso = cdf(root_path, 'Orejas_de_oso',4,on_ram)
        df_Otros = cdf(root_path, 'Otros',5,on_ram)
        
        self.Dataframe = pd.concat([df_Rosas, df_Calas_rosa,df_Cardenales_rojas,df_Orejas_de_oso,df_Otros]
                                       , ignore_index=True)
        
        if shuffle:
            self.Dataframe = self.Dataframe.sample(frac=1).reset_index(drop=True)
        
        
        
        
        
        pass
    
    def __len__(self):
        return len(self.Dataframe.index)
    
    def __getitem__(self, idx):
        
        row = self.Dataframe.iloc[[0]]
        
        numero = row.values[0][1]
        #print(numero)
        
        label = torch.tensor(int(numero)) 
        
        
        if self.on_ram:
            img = row.values[0][0]
        else:
            img = cv2.imread(row.values[0][0])
        #img = cv2.imread(img_path)
        
        if self.transform:
            img = self.transform(img)
        
        
        return (img,label)

## CIFAR dataset

In [4]:
transform = transforms.Compose([
    torchvision.transforms.ToPILImage(mode=None),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = Dataset_Flores(root_path=os.path.join(os.getcwd(),"Dataset"),on_ram = False,test=False, transform=transform)
val_ds = Dataset_Flores(root_path=os.path.join(os.getcwd(),"Dataset"),on_ram = False,test=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=16, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=16, num_workers=4)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(f'train len {len(train_ds)}')
print(f'val len {len(val_ds)}')

train len 692
val len 171


## Important concepts
- Fine tuning

In [5]:
class CIFARModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.alexnet(pretrained=True)
        print(self.model)
        self.model.classifier[4] = torch.nn.Linear(4096,1024)
        self.model.classifier[6] = torch.nn.Linear(1024,5)
        print(self.model)
        self.val_acc = torchmetrics.Accuracy(num_classes=5)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        out = self(x)
        loss = F.cross_entropy(out, y)
        print(out.size())
        self.val_acc(out, y.long())
        self.log('val_acc', self.val_acc, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_loss", loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [6]:
# Init our model
model = CIFARModel()
logger = TensorBoardLogger('./log')
checkpoint_callback = ModelCheckpoint(dirpath='./log/checkpoints',
                                      save_top_k=1,
                                      verbose=True,
                                      monitor='val_loss',
                                      mode='min')

# Initialize a trainer
trainer = Trainer(
    accelerator='gpu',
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback]
)

# Train the model
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\varga\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:611: UserWarning: Checkpoint directory C:\Users\varga\OneDrive\Escritorio\Universidad-Semestre Actual\Computer Vision\29\log\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 


  | Name    | Type     | Params
-------------------------------------
0 | model   | AlexNet  | 44.4 M
1 | val_acc | Accuracy | 0     
-------------------------------------
44.4 M    Trainable params
0         Non-trainable params
44.4 M    Total params
177.692   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\varga\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: DataLoader worker (pid(s) 19452) exited unexpectedly

In [ ]:
train_ds.__getitem__(0)[0].shape